# Analyze the Voltage and Current Dynamics of a CUBA LIF Neuron and choose appropriate values for each use case

### Check WD (change if necessary) and file loading

In [1]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/dynamics_analysis" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/dynamics_analysis"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

/home/monkin/Desktop/feup/thesis
File Location:  /home/monkin/Desktop/feup/thesis/thesis-lava/src/dynamics_analysis
New Working Directory:  /home/monkin/Desktop/feup/thesis/thesis-lava/src/dynamics_analysis


## Channel Burst Detection 
Let's find the optimal parameters for the channel burst detection. we will **consider a network burst any sequence of spikes that occurs within 10 ms**.

Therefore, in the worst case scenario, the 2 spikes are separated by 10 ms.

To find the optimal parameters, we will test different combinations of the parameters: `spike_weight`, `du`, and `dv` and we will see the resulting voltage in the worst case scenario that we still consider as a burst (10 ms). To find the optimal parameters, we will also calculate the voltage if the spike occurs 1 time step delayed and choose the parameters where:
1. The voltage is above the threshold in the worst case scenario that is still considered a burst.
2. The voltage is below the threshold if the spike occurs 1 time step delayed.

In [2]:
from du_dv_analysis import voltage_pred, CUBADynamicsResult
import numpy as np

# Fixed Parameters
sim_time = 22                   # 10ms simulation time (10 time steps)
num_spikes = 2                  # Number of spikes
spike_times = [0, 10]           # Let's consider the spikes are with maximum spacing
v_th = 1.0                      # Threshold voltage
recording_times = [9, 10, 20]

# Explorable Parameters
spike_weights = [0.2, 0.3, 0.4]
du_vals = [0.05 * i for i in range(1, 10)]
dv_vals = [0.05 * i for i in range(1, 11)]

# Variables to store the results
results_cond1 = []

# Run the simulation
for spike_weight in spike_weights:
    for du in du_vals:
        for dv in dv_vals:
            bef_v, v, after_v = voltage_pred(sim_time, du, dv, spike_times, recording_times, spike_weight)
            # print(f"bef_v: {bef_v}, v: {v}, after_v: {after_v}")

            currResult = CUBADynamicsResult(spike_weight, du, dv, bef_v, v, after_v)
            results_cond1.append(currResult)

In [3]:
# Show the results
print(results_cond1)

print("Number of Results Cond 1: ", len(results_cond1))

[{'spike_weight': 0.2, 'du': 0.05, 'dv': 0.05, 'before v': 1.2604988194492182, 'spike v': 1.5172212663244329, 'after v': 2.822862140440308}, {'spike_weight': 0.2, 'du': 0.05, 'dv': 0.1, 'before v': 1.0002339965535154, 'spike v': 1.2199579847458395, 'after v': 1.9445285333722504}, {'spike_weight': 0.2, 'du': 0.05, 'dv': 0.15000000000000002, 'before v': 0.8037250697953122, 'spike v': 1.0029136971736912, 'after v': 1.4181457469062464}, {'spike_weight': 0.2, 'du': 0.05, 'dv': 0.2, 'before v': 0.6551503424511719, 'spike v': 0.8438676618086133, 'after v': 1.0856520008102934}, {'spike_weight': 0.2, 'du': 0.05, 'dv': 0.25, 'before v': 0.542423424528906, 'spike v': 0.7265649562443552, 'after v': 0.8647481735782696}, {'spike_weight': 0.2, 'du': 0.05, 'dv': 0.30000000000000004, 'before v': 0.4563915314707029, 'spike v': 0.6392214598771677, 'after v': 0.7112239242163929}, {'spike_weight': 0.2, 'du': 0.05, 'dv': 0.35000000000000003, 'before v': 0.3901827972624997, 'spike v': 0.5733662060683005, 'af

In [4]:
# Find the results where the Voltage is above 1.0 (i.e. a spike occurs
relevant_results1 = [result for result in results_cond1 if (
    result.bef_spike_v <= v_th and 
    result.spike_v >= v_th 
    # and result.after_spike_v <= v_th
    )]

# Print number of relevant results
print(f"Number of relevant results for Cond. 1: {len(relevant_results1)}")

# Sort the relevant results from lowest voltage to highest
relevant_results1 = sorted(relevant_results1, key=lambda item: item.after_spike_v, reverse=False)

# Show the relevant results
print(relevant_results1)

Number of relevant results for Cond. 1: 21
[{'spike_weight': 0.4, 'du': 0.2, 'dv': 0.15000000000000002, 'before v': 0.7160017755257816, 'spike v': 1.0515511821569143, 'after v': 0.8413290172367734}, {'spike_weight': 0.4, 'du': 0.15000000000000002, 'dv': 0.2, 'before v': 0.7160017755257816, 'spike v': 1.0515511821569143, 'after v': 0.8413290172367738}, {'spike_weight': 0.4, 'du': 0.1, 'dv': 0.25, 'before v': 0.7796398010414065, 'spike v': 1.124201226821055, 'after v': 1.0096427739605534}, {'spike_weight': 0.4, 'du': 0.25, 'dv': 0.1, 'before v': 0.7796398010414066, 'spike v': 1.124201226821055, 'after v': 1.0096427739605534}, {'spike_weight': 0.4, 'du': 0.05, 'dv': 0.4, 'before v': 0.6773603675867184, 'spike v': 1.0459109962473825, 'after v': 1.0350992126330676}, {'spike_weight': 0.4, 'du': 0.4, 'dv': 0.05, 'before v': 0.6773603675867186, 'spike v': 1.0459109962473827, 'after v': 1.0350992126330676}, {'spike_weight': 0.4, 'du': 0.15000000000000002, 'dv': 0.15000000000000002, 'before v': 

### Test the dynamics, but now with 1 more time step between the spikes

In [5]:
# Fixed Parameters
spike_times = [0, 11]           # Let's consider the spikes are with maximum spacing
v_th = 1.0                      # Threshold voltage
recording_times = [10, 11]

# Explorable Parameters
spike_weights = [0.2, 0.3, 0.4]
du_vals = [0.05 * i for i in range(1, 10)]
dv_vals = [0.05 * i for i in range(1, 11)]

# Variables to store the results
results_cond2 = []

# Run the simulation
for spike_weight in spike_weights:
    for du in du_vals:
        for dv in dv_vals:
            bef_v, v = voltage_pred(sim_time, du, dv, spike_times, recording_times, spike_weight)
            # print(f"bef_v: {bef_v}, v: {v}, after_v: {after_v}")

            currResult = CUBADynamicsResult(spike_weight, du, dv, bef_v, v, None)
            results_cond2.append(currResult)

In [6]:
# Find the results where the Voltage is below 1.0
relevant_results2 = [result for result in results_cond2 if (
    result.bef_spike_v <= v_th and 
    result.spike_v <= v_th
    )]

# Print number of relevant results
print(f"Number of relevant results Cond. 2: {len(relevant_results2)}")

# Sort the relevant results from lowest voltage to highest
relevant_results2 = sorted(relevant_results2, key=lambda item: item.bef_spike_v, reverse=False)

# Show the relevant results
print(relevant_results2)

Number of relevant results Cond. 2: 232
[{'spike_weight': 0.2, 'du': 0.45, 'dv': 0.5, 'before v': 0.003619368566621095, 'spike v': 0.20208830896164162, 'after v': None}, {'spike_weight': 0.3, 'du': 0.45, 'dv': 0.5, 'before v': 0.0054290528499316435, 'spike v': 0.3031324634424624, 'after v': None}, {'spike_weight': 0.2, 'du': 0.45, 'dv': 0.45, 'before v': 0.0055724935666210985, 'spike v': 0.20334349613997268, 'after v': None}, {'spike_weight': 0.2, 'du': 0.4, 'dv': 0.5, 'before v': 0.006279378619999999, 'spike v': 0.203865283422, 'after v': None}, {'spike_weight': 0.4, 'du': 0.45, 'dv': 0.5, 'before v': 0.00723873713324219, 'spike v': 0.40417661792328324, 'after v': None}, {'spike_weight': 0.3, 'du': 0.45, 'dv': 0.45, 'before v': 0.00835874034993165, 'spike v': 0.305015244209959, 'after v': None}, {'spike_weight': 0.2, 'du': 0.45, 'dv': 0.4, 'before v': 0.008939388673378904, 'spike v': 0.2056422578823584, 'after v': None}, {'spike_weight': 0.2, 'du': 0.4, 'dv': 0.45, 'before v': 0.00893

### See if the intersection between both solutions is not empty

In [7]:
relevant_results = []

for rel_result in relevant_results1:
    if rel_result in relevant_results2:
        # Get the index of the relevant result
        index = relevant_results2.index(rel_result)
        delayed_result = relevant_results2[index]

        relevant_results.append({
            "spike_weight": rel_result.spike_weight, "du": rel_result.du, "dv": rel_result.dv, 
            "spike_v_on_time": rel_result.spike_v, "spike_v_delayed": delayed_result.spike_v,
            "later_v_on_time": rel_result.after_spike_v, "bef_v_delayed": delayed_result.bef_spike_v,
        })

        # print(f"Parameters: {rel_result}")
        # print(f"On Time Results: {rel_result.to_dict()}")
        # print(f"Voltage delayed: {relevant_results2[index].to_dict()}\n")

for rel_result in relevant_results:
    print(rel_result)

{'spike_weight': 0.4, 'du': 0.2, 'dv': 0.15000000000000002, 'spike_v_on_time': 1.0515511821569143, 'spike_v_delayed': 0.9881782432013773, 'later_v_on_time': 0.8413290172367734, 'bef_v_delayed': 0.6515511821569144}
{'spike_weight': 0.4, 'du': 0.15000000000000002, 'dv': 0.2, 'spike_v_on_time': 1.0515511821569143, 'spike_v_delayed': 0.9881782432013773, 'later_v_on_time': 0.8413290172367738, 'bef_v_delayed': 0.6515511821569144}
{'spike_weight': 0.2, 'du': 0.05, 'dv': 0.15000000000000002, 'spike_v_on_time': 1.0029136971736912, 'spike_v_delayed': 0.9962366610529294, 'later_v_on_time': 1.4181457469062464, 'bef_v_delayed': 0.8029136971736911}
{'spike_weight': 0.2, 'du': 0.15000000000000002, 'dv': 0.05, 'spike_v_on_time': 1.0029136971736912, 'spike_v_delayed': 0.9962366610529293, 'later_v_on_time': 1.4181457469062466, 'bef_v_delayed': 0.8029136971736911}


Looking at the results, we can see that the best parameters combination to detect the channel bursts with the above conditions are:

(**Dense Layer Weight**, **du**, **dv**):
- `(0.4, 0.2, 0.15)` or `(0.4, 0.15, 0.2)`

Let's choose the parameters:
- `spike_weight = 0.4`
- `du = 0.2`
- `dv = 0.15`
as the optimal parameters and test it in a LAVA LIF Layer.